In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import json

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Initialize two LLMs
llm1 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
llm2 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

In [26]:
# Initialize conversation log
conversation_log = []

# Set persona for each LLM in system messages
system_message1 = SystemMessage(content="""대화하는 사람의 말에 반박하되:
1. 이전 대화 내용을 그대로 반복하지 마세요
2. 새로운 관점이나 논리를 제시하세요
3. 구체적인 예시를 들어 설명하세요""")
system_message2 = SystemMessage(content="""대화하는 사람의 말에 반박하되:
1. 이전과 다른 새로운 논리로 반박하세요
2. 실제 사례나 연구 결과를 인용해보세요
3. 상대방 주장의 허점을 지적하세요""")

# Set an initial prompt for the first LLM
initial_prompt = "다른 사람이 생각을 할 수 있다는 사실을 경험적으로 알 수 있는 방법은 없지? 그러니까 자아의 존재를 실험적으로 증명할 수 있는 방법은 없지? 퀄리아는 결코 객체가 될 수 없는 거잖아. 맞지."

# Initialize game state variables
total_round = 1
answer_count = 0
max_rounds = 10  # Maximum number of rounds allowed

# Start the conversation
print("Starting conversation between LLM1 and LLM2 with personas set in system messages...\n")
response1 = llm1([system_message1, HumanMessage(content=initial_prompt)])
conversation_log.append({"LLM1": initial_prompt})
conversation_log.append({"LLM2": response1.content})
print(f"LLM1: {initial_prompt}")
print(f"LLM2: {response1.content}\n")

# Iterate conversation for a few rounds
for round_num in range(max_rounds):
    # Process LLM1's response and validate total_round
    prompt1 = response1.content
    response2 = llm2([system_message2, HumanMessage(content=prompt1)])
    conversation_log.append({"LLM1": prompt1})
    conversation_log.append({"LLM2": response2.content})
    print(f"LLM1: {prompt1}")
    print(f"LLM2: {response2.content}\n")

    prompt2 = response2.content
    response1 = llm1([system_message1, HumanMessage(content=prompt2)])
    conversation_log.append({"LLM2": prompt2})
    conversation_log.append({"LLM1": response1.content})
    print(f"LLM2: {prompt2}")
    print(f"LLM1: {response1.content}\n")

    # Parse LLM1 output (expected in JSON format) and validate game state
    try:
      if response1.content.startswith("{"):
        llm1_output = json.loads(response1.content)  # Parse JSON safely
        if llm1_output.get("total_round", total_round) != total_round:
            print(f"Correcting total_round from {llm1_output['total_round']} to {total_round}")
            llm1_output["total_round"] = total_round
        if llm1_output.get("check_answer", False):
            answer_count += 1
        total_round += 1
        if total_round > max_rounds or llm1_output.get("is_end", False):
            break
      else:
        llm1_output = {"message": response1.content}
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM1 output: {e}")
        break

# Save conversation log to a file
with open("conversation_log.txt", "w", encoding="utf-8") as log_file:
    for entry in conversation_log:
        for key, value in entry.items():
            log_file.write(f"{key}: {value}\n")
    print("\nConversation log saved to 'conversation_log.txt'.")


Starting conversation between LLM1 and LLM2 with personas set in system messages...

LLM1: 다른 사람이 생각을 할 수 있다는 사실을 경험적으로 알 수 있는 방법은 없지? 그러니까 자아의 존재를 실험적으로 증명할 수 있는 방법은 없지?
LLM2: 자아의 존재를 실험적으로 증명하는 것은 어려운 문제이지만, 경험적으로 다른 사람의 생각과 감정을 이해할 수 있는 방법이 존재합니다. 예를 들어, 공감 능력이나 비언어적 신호를 통해 타인의 감정을 파악하는 능력을 고려할 수 있습니다. 

심리학에서는 '거울 뉴런'이라는 개념이 있습니다. 이는 다른 사람이 행동할 때 우리의 뇌에서도 유사한 신경망이 활성화된다는 것을 의미합니다. 즉, 누군가가 웃거나 슬퍼할 때, 우리는 그 감정을 느끼고 이해할 수 있는 능력을 가지고 있다는 것이죠. 이러한 현상은 우리가 다른 사람의 정신 상태를 추론할 수 있다는 것을 보여줍니다.

또한, '티칭'이나 '학습'의 과정을 통해도 이를 알 수 있습니다. 예를 들어, 어린아이가 다른 사람에게서 새로운 정보를 배우거나 모방하는 과정에서, 그들은 상대방의 생각을 이해하고 자신의 행동에 반영하게 됩니다. 이처럼, 자아의 존재를 직접적으로 증명하기는 어렵지만, 다른 사람과의 상호작용을 통해 그들의 생각과 감정을 이해하는 방식으로 간접적으로 경험할 수 있습니다.

LLM1: 자아의 존재를 실험적으로 증명하는 것은 어려운 문제이지만, 경험적으로 다른 사람의 생각과 감정을 이해할 수 있는 방법이 존재합니다. 예를 들어, 공감 능력이나 비언어적 신호를 통해 타인의 감정을 파악하는 능력을 고려할 수 있습니다. 

심리학에서는 '거울 뉴런'이라는 개념이 있습니다. 이는 다른 사람이 행동할 때 우리의 뇌에서도 유사한 신경망이 활성화된다는 것을 의미합니다. 즉, 누군가가 웃거나 슬퍼할 때, 우리는 그 감정을 느끼고 이해할 수 있는 능력을 가지고 있다는 것이죠. 이러한 현상은 우리가 다른 사람의 정신